# Steam 게임 상세정보 수집 및 결측값 보완

## 조건
- 입력 데이터 : 02-3. steam_data_real.csv
- 수집 대상 : 개발사, 배급사, 가격
- 데이터 소스
    - 1차: Steam API
    - 2차: Selenium 크롤링 (스토어 페이지 직접 접속)
    - 3차: API 재시도 (결측값이 계속 생겼을 시)
- API 수집 시 500개 단위 중간 저장

## 절차
- AppID 목록 준비 : 02-3. steam_data_real.csv에서 appid 열을 리스트로 추출
- 1차 수집 (Steam API)
    - 각 AppID별 개발사, 배급사, 가격 수집
    - 500개 단위로 부분 저장
- 결측값 식별
    - 개발사, 배급사, 가격 중 하나라도 NaN인 행 필터링 → missinglist 생성
- 2차 수집 (Selenium 크롤링)
    - missinglist의 AppID별 스토어 페이지 접속
    - 개발사, 배급사, 가격 추출
- 병합 및 결합
    - 1차 수집 결과와 2차 크롤링 결과를 AppID 기준 병합
- 3차 보완 (API 재시도)
- 중복/결측 처리

In [20]:
import pandas as pd
game_df = pd.read_csv('02-3. steam_data_real.csv', encoding='utf-8-sig')

In [21]:
game_df.head(5)

,appid,게임명,출시일,장르,OS,DLC여부,평가점수,사용자 평가 수
0,1384160,guilty gear -strive-,2021년 06월 11일,격투 | 2D 격투 | 웅장한 사운드트랙 | 애니메이션 | PvP,Windows,0,평가 점수: 매우 긍정적,"사용자 평가 43,066개"
1,2141910,magic: the gathering arena,2023년 05월 24일,전략 | 카드 게임 | 트레이딩 카드 게임 | 무료 플레이 | 덱빌딩,Windows,0,평가 점수: 복합적,"사용자 평가 21,004개"
2,1477590,ez2on reboot : r,2022년 04월 14일,리듬 | 액션 | 음악 | 캐주얼 | 스포츠,Windows,0,평가 점수: 매우 긍정적,"사용자 평가 5,612개"
3,1326470,sons of the forest,2024년 02월 23일,생존 | 오픈 월드 | 멀티플레이어 | 협동 | 생존 공포,Windows,0,평가 점수: 매우 긍정적,"사용자 평가 234,109개"
4,550,left 4 dead 2,2009년 11월 17일,좀비 | 협동 | 1인칭 슈팅 | 멀티플레이어 | 슈팅,Windows,0,평가 점수: 압도적으로 긍정적,"사용자 평가 747,304개"


In [22]:
# appid 열을 리스트로 변환
appid_list = game_df['appid'].tolist()
print(appid_list[:10])

[1384160, 2141910, 1477590, 1326470, 550, 477160, 2290180, 739630, 1677280, 2157560]


### 1. 1차 수집

In [ ]:
import requests
import pandas as pd
import numpy as np

# 예시: game_df["appid"] 열에서 리스트 추출
appid_list = game_df["appid"].dropna().astype(int).tolist()

developers = []
publishers = []
prices = []

total = len(appid_list)
save_interval = 500

for idx, appid in enumerate(appid_list, 1):
    try:
        url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
        response = requests.get(url, timeout=5)
        data = response.json()

        # 안전한 데이터 접근
        app_data = data.get(str(appid))

        if (app_data is not None and 
            app_data.get('success') and 
            app_data.get('data') is not None):

            info = app_data['data']

            dev = ", ".join(info.get('developers', []))
            pub = ", ".join(info.get('publishers', []))
            price_overview = info.get('price_overview')
            price = price_overview.get('final_formatted', '무료 또는 정보 없음') if price_overview else '무료 또는 정보 없음'

            developers.append(dev if dev else np.nan)
            publishers.append(pub if pub else np.nan)
            prices.append(price)

            print(f"[{idx}/{total}] appid: {appid} | 개발사: {dev} | 배급사: {pub} | 가격: {price}")
        else:
            developers.append(np.nan)
            publishers.append(np.nan)
            prices.append(np.nan)
            print(f"[{idx}/{total}] appid: {appid} | ❌ 정보 없음")

    except Exception as e:
        print(f"⚠️ appid {appid} 에러 발생: {e}")
        developers.append(np.nan)
        publishers.append(np.nan)
        prices.append(np.nan)

    # 500개마다 저장
    if idx % save_interval == 0:
        temp_df = game_df.iloc[:idx].copy()
        temp_df["개발사"] = developers
        temp_df["배급사"] = publishers
        temp_df["가격"] = prices
        temp_df.to_csv(f"steam_game_info_partial_{idx}.csv", index=False)
        print(f"💾 {idx}개 저장 완료 (steam_game_info_partial_{idx}.csv)")

# 최종 결과 저장
game_df = game_df.copy()
game_df["개발사"] = developers
game_df["배급사"] = publishers
game_df["가격"] = prices
game_df.to_csv("03-2. steam_game_info_final.csv", index=False)

print("\n✅ 게임 정보 수집 완료 및 최종 저장 완료 (03-2. steam_game_info_final.csv)")
print(game_df.head())


[1/19426] appid: 1384160 | 개발사: Arc System Works | 배급사: Arc System Works | 가격: ₩ 19,950
[2/19426] appid: 2141910 | 개발사: Wizards of the Coast LLC | 배급사: Wizards of the Coast LLC | 가격: 무료 또는 정보 없음
[3/19426] appid: 1477590 | 개발사: Neonovice Co., Ltd., SQUARE PIXELS | 배급사: Neonovice Co., Ltd. | 가격: ₩ 44,800
[4/19426] appid: 1326470 | 개발사: Endnight Games Ltd | 배급사: Newnight  | 가격: ₩ 32,000
[5/19426] appid: 550 | 개발사: Valve | 배급사: Valve | 가격: ₩ 11,000
[6/19426] appid: 477160 | 개발사: No Brakes Games | 배급사: Curve Games | 가격: 21,59zł
[7/19426] appid: 2290180 | 개발사: Ubisoft Annecy, Ubisoft Belgrade, Ubisoft Kyiv, Ubisoft Pune, Ubisoft Odesa | 배급사: Ubisoft Entertainment | 가격: ₩ 4,550
[8/19426] appid: 739630 | 개발사: Kinetic Games | 배급사: Kinetic Games | 가격: ₩ 21,500
[9/19426] appid: 1677280 | 개발사: Relic Entertainment | 배급사: Relic Entertainment | 가격: ₩ 59,800
[10/19426] appid: 2157560 | 개발사: Cygames, Inc., Arc System Works | 배급사: Cygames, Inc. | 가격: ₩ 21,370
[11/19426] appid: 2617700 | 개발사: CodeManu, E

In [27]:
game_df.to_csv('03-1. steam_data_추가.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 셋 중 하나라도 NaN인 행만 필터링
missing_info_df = game_df[
    game_df[['개발사', '배급사', '가격']].isna().any(axis=1)]

# 결과 확인
print(f"🔍 정보가 누락된 게임 수: {len(missing_info_df)}개")
print(missing_info_df[['appid', '게임명', '개발사', '배급사', '가격']].head())

🔍 정보가 누락된 게임 수: 15194개
       appid                                        게임명  \
42   3102940                           ea sports fc™ 25   
54    401920           the binding of isaac: afterbirth   
232  3796620  dead by daylight: five nights at freddy’s   
248  1305420                                      agrou   
249  1601330                           survival machine   

                                개발사  배급사        가격  
42                              NaN  NaN       NaN  
54   Nicalis, Inc., Edmund McMillen  NaN  ₩ 11,500  
232      Behaviour Interactive Inc.  NaN   ₩ 5,500  
248                             NaN  NaN       NaN  
249                             NaN  NaN       NaN  


In [29]:
missinglist = missing_info_df['appid'].tolist()

In [32]:
missinglist

[3102940,
 401920,
 3796620,
 1305420,
 1601330,
 1307550,
 1374490,
 1995690,
 784080,
 3144890,
 692890,
 3300460,
 973580,
 1304930,
 1479730,
 967050,
 229480,
 42680,
 115300,
 3787460,
 3645890,
 526160,
 780290,
 677160,
 2269950,
 1284210,
 95400,
 1928870,
 1058450,
 2132850,
 311690,
 252110,
 1722840,
 1167450,
 1799930,
 2780470,
 1049320,
 1302240,
 2774380,
 3408110,
 2780980,
 1548850,
 1307890,
 633230,
 3168600,
 661920,
 1937780,
 3511130,
 1522820,
 1695840,
 218620,
 384180,
 3249650,
 1939350,
 207140,
 846800,
 1106840,
 1227650,
 535520,
 2698780,
 2273980,
 1677740,
 3453600,
 3030360,
 2438160,
 2216360,
 250180,
 1773180,
 971690,
 1708850,
 922500,
 2736340,
 1140620,
 2290000,
 891360,
 1206200,
 10090,
 3198360,
 3453970,
 3484300,
 1599330,
 515180,
 2438990,
 874390,
 302510,
 312610,
 233610,
 550340,
 1007040,
 1194630,
 2439690,
 460930,
 348550,
 1088790,
 427730,
 680420,
 547360,
 1729140,
 204100,
 1018020,
 647830,
 555880,
 2768430,
 1105420,
 66

### 2. 결측값 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

# 크롬 드라이버 설정
options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=options)

# missinglist: appid 리스트
# 예시: missinglist = [123456, 789101, ...]
# 미리 숫자로 형변환 (혹시 문자열일 경우 대비)
missinglist = [int(a) for a in missinglist]

# 결과 저장 리스트
developers = []
publishers = []
prices = []
collected_ids = []

# 크롤링 시작
for idx, appid in enumerate(missinglist, 1):
    url = f"https://store.steampowered.com/app/{appid}"
    driver.get(url)
    time.sleep(2)

    print(f"[{idx}/{len(missinglist)}] 🔍 appid: {appid}")

    try:
        # 개발사
        try:
            dev = driver.find_element(By.ID, "developers_list").text.strip()
        except:
            dev = None

        # 배급사
        try:
            dev_rows = driver.find_elements(By.CLASS_NAME, "dev_row")
            pub = None
            for row in dev_rows:
                summary = row.find_element(By.CLASS_NAME, "summary")
                if summary.get_attribute("id") != "developers_list":
                    pub = summary.text.strip()
                    break
        except:
            pub = None

        # 가격
        try:
            price = driver.find_element(By.CLASS_NAME, "game_purchase_price").text.strip()
        except:
            price = "정보 없음"

        developers.append(dev)
        publishers.append(pub)
        prices.append(price)
        collected_ids.append(appid)

        print(f"개발사: {dev} | 배급사: {pub} | 가격: {price}")

    except Exception as e:
        print(f"오류 발생: {e}")
        developers.append(None)
        publishers.append(None)
        prices.append(None)
        collected_ids.append(appid)

driver.quit()

# 데이터프레임 생성
result_df = pd.DataFrame({
    'appid': collected_ids,
    '개발사_크롤링': developers,
    '배급사_크롤링': publishers,
    '가격_크롤링': prices
})

# 결과 확인
print("\n 크롤링 결과:")
print(result_df.head())

# 저장
result_df.to_csv("steam_missing_info_crawled.csv", index=False, encoding="utf-8-sig")

In [1]:
import pandas as pd

### 3. 간단한 정제 후 맵핑

In [23]:
steam_eng = pd.read_csv("00-2. 스팀게임목록_영어.csv", encoding="utf-8-sig")
steam_plus = pd.read_csv("03-1. steam_data_추가.csv", encoding="utf-8-sig")
steam_missing = pd.read_csv("03-2. steam_missing_info_crawled.csv", encoding="utf-8-sig")

In [24]:
steam_eng['평가점수'] = steam_eng['평가점수'].str.replace('평가 점수: ', '', regex=False)

In [25]:
steam_eng['사용자 평가 수'] = steam_eng['사용자 평가 수'].str.replace('사용자 평가 ', '', regex=False)

In [26]:
steam_eng

,appid,게임명,출시일,장르,OS,DLC여부,평가점수,사용자 평가 수
0,1384160,guilty gear -strive-,2021년 06월 11일,격투 | 2D 격투 | 웅장한 사운드트랙 | 애니메이션 | PvP,Windows,0,매우 긍정적,"43,066개"
1,2141910,magic: the gathering arena,2023년 05월 24일,전략 | 카드 게임 | 트레이딩 카드 게임 | 무료 플레이 | 덱빌딩,Windows,0,복합적,"21,004개"
2,1477590,ez2on reboot : r,2022년 04월 14일,리듬 | 액션 | 음악 | 캐주얼 | 스포츠,Windows,0,매우 긍정적,"5,612개"
3,1326470,sons of the forest,2024년 02월 23일,생존 | 오픈 월드 | 멀티플레이어 | 협동 | 생존 공포,Windows,0,매우 긍정적,"234,109개"
4,550,left 4 dead 2,2009년 11월 17일,좀비 | 협동 | 1인칭 슈팅 | 멀티플레이어 | 슈팅,Windows,0,압도적으로 긍정적,"747,304개"
...,...,...,...,...,...,...,...,...
19331,1723880,crabs and rats,출시 예정,액션 | 캐주얼 | 아케이드 | 플랫폼 | 2D 플랫폼,Windows,0,NaN,NaN
19332,1782170,adventure jam,출시 예정,어드벤처 | 액션 | RPG | 2D 격투 | 액션 어드벤처,Windows,0,NaN,NaN
19333,346110,the train arrived!,발표 예정,어드벤처 | RPG | 액션 | 기차 | 퍼즐,Windows,0,NaN,NaN
19334,3010460,dispatch control,발표 예정,시뮬레이션 | 전략 | 캐주얼 | 실시간 전략 | 대전략,Windows,0,NaN,NaN


In [27]:
steam_missing = steam_missing.rename(columns={
    '개발사_크롤링': '개발사',
    '배급사_크롤링': '배급사',
    '가격_크롤링': '가격'
})

In [28]:
# 1. steam_plus에서 appid 기준으로 개발사, 배급사, 가격 붙이기
steam_eng = pd.merge(
    steam_eng,
    steam_plus[['appid', '개발사', '배급사', '가격']],
    on='appid',
    how='left'
)

# 2. steam_missing에서 appid 기준으로 추가 정보 붙이기 (없는 값만 채우기)
steam_eng = pd.merge(
    steam_eng,
    steam_missing[['appid', '개발사', '배급사', '가격']],
    on='appid',
    how='left',
    suffixes=('', '_missing')
)

# 3. NaN 값은 steam_missing에서 가져오기
for col in ['개발사', '배급사', '가격']:
    steam_eng[col] = steam_eng[col].combine_first(steam_eng[f'{col}_missing'])
    steam_eng = steam_eng.drop(columns=[f'{col}_missing'])

# 결과 확인
steam_eng.head()

,appid,게임명,출시일,장르,OS,DLC여부,평가점수,사용자 평가 수,개발사,배급사,가격
0,1384160,guilty gear -strive-,2021년 06월 11일,격투 | 2D 격투 | 웅장한 사운드트랙 | 애니메이션 | PvP,Windows,0,매우 긍정적,"43,066개",Arc System Works,Arc System Works,"₩ 19,950"
1,2141910,magic: the gathering arena,2023년 05월 24일,전략 | 카드 게임 | 트레이딩 카드 게임 | 무료 플레이 | 덱빌딩,Windows,0,복합적,"21,004개",Wizards of the Coast LLC,Wizards of the Coast LLC,무료 또는 정보 없음
2,1477590,ez2on reboot : r,2022년 04월 14일,리듬 | 액션 | 음악 | 캐주얼 | 스포츠,Windows,0,매우 긍정적,"5,612개","Neonovice Co., Ltd., SQUARE PIXELS","Neonovice Co., Ltd.","₩ 44,800"
3,1477590,ez2on reboot : r,2022년 04월 14일,리듬 | 액션 | 음악 | 캐주얼 | 스포츠,Windows,0,매우 긍정적,"5,612개","Neonovice Co., Ltd., SQUARE PIXELS","Neonovice Co., Ltd.","₩ 44,800"
4,1326470,sons of the forest,2024년 02월 23일,생존 | 오픈 월드 | 멀티플레이어 | 협동 | 생존 공포,Windows,0,매우 긍정적,"234,109개",Endnight Games Ltd,Newnight,"₩ 32,000"


In [29]:
recrawled = steam_eng[steam_eng['가격'].isnull()][['appid']]

In [30]:
recrawled = recrawled['appid'].tolist()

### 4. 누락값 3차 보완

In [31]:
import requests
import pandas as pd
import numpy as np
import time

# 🔹 전제: steam_eng와 recrawled 리스트가 준비되어 있어야 함
# 예시:
# steam_eng = pd.read_csv("steam_eng.csv")
# recrawled = [int(x) for x in steam_eng[steam_eng["개발사"].isna()]["appid"].tolist()]

# 🔸 결과 저장 리스트
developers = []
publishers = []
prices = []

total = len(recrawled)
save_interval = 500

for idx, appid in enumerate(recrawled, 1):
    try:
        url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
        response = requests.get(url, timeout=5)
        
        try:
            data = response.json()
        except Exception as e:
            print(f"[{idx}/{total}] ❌ JSON 파싱 실패 | appid: {appid}")
            developers.append(np.nan)
            publishers.append(np.nan)
            prices.append(np.nan)
            continue

        app_data = data.get(str(appid))
        if app_data and app_data.get("success") and app_data.get("data"):
            info = app_data["data"]
            dev = ", ".join(info.get("developers", [])) or np.nan
            pub = ", ".join(info.get("publishers", [])) or np.nan
            price_overview = info.get("price_overview")
            price = price_overview.get("final_formatted", "무료 또는 정보 없음") if price_overview else "무료 또는 정보 없음"

            print(f"[{idx}/{total}] ✅ appid: {appid} | 개발사: {dev} | 배급사: {pub} | 가격: {price}")
        else:
            dev, pub, price = np.nan, np.nan, np.nan
            print(f"[{idx}/{total}] ❌ 정보 없음 | appid: {appid}")

        developers.append(dev)
        publishers.append(pub)
        prices.append(price)

    except Exception as e:
        print(f"[{idx}/{total}] ⚠️ 예외 발생 | appid: {appid} | {e}")
        developers.append(np.nan)
        publishers.append(np.nan)
        prices.append(np.nan)

    time.sleep(0.3)  # 과도한 요청 방지

# 🔸 1. 크롤링 결과 DataFrame 생성
result_df = pd.DataFrame({
    "appid": recrawled,
    "개발사_수정": developers,
    "배급사_수정": publishers,
    "가격_수정": prices
})

# 🔸 2. steam_eng에 병합
steam_eng = steam_eng.merge(result_df, on="appid", how="left")

# 🔸 3. NaN 값만 조건부 업데이트
steam_eng["개발사"] = steam_eng["개발사"].fillna(steam_eng["개발사_수정"])
steam_eng["배급사"] = steam_eng["배급사"].fillna(steam_eng["배급사_수정"])
steam_eng["가격"] = steam_eng["가격"].fillna(steam_eng["가격_수정"])

# 🔸 4. 임시 컬럼 제거
steam_eng.drop(columns=["개발사_수정", "배급사_수정", "가격_수정"], inplace=True)

# 🔸 5. 저장
steam_eng.to_csv("03-3. steam_eng_updated.csv", index=False, encoding="utf-8-sig")
print("\n✅ 업데이트 완료 및 저장됨 → 03-3. steam_eng_updated.csv")


[1/44] ✅ appid: 782330 | 개발사: id Software | 배급사: Bethesda Softworks | 가격: CDN$ 13.37
[2/44] ✅ appid: 520440 | 개발사: Arc System Works | 배급사: Arc System Works | 가격: ₩ 10,560
[3/44] ✅ appid: 801630 | 개발사: FRENCH-BREAD | 배급사: Arc System Works | 가격: ₩ 8,200
[4/44] ✅ appid: 1222300 | 개발사: Fully Involved Game Studios | 배급사: PlayWay S.A. | 가격: ₩ 3,890
[5/44] ✅ appid: 895400 | 개발사: Bad Pixel | 배급사: Bad Pixel, tinyBuild | 가격: ₩ 16,200
[6/44] ✅ appid: 2324300 | 개발사: CrazyLabs | 배급사: QubicGames, Gamersky Games | 가격: ₩ 1,120
[7/44] ✅ appid: 1214650 | 개발사: Question | 배급사: THQ Nordic | 가격: ₩ 12,800
[8/44] ✅ appid: 1158940 | 개발사: Get Up Games, Iceberg Interactive | 배급사: Iceberg Interactive | 가격: ₩ 4,950
[9/44] ✅ appid: 2017080 | 개발사: Fair Play Labs | 배급사: GameMill Entertainment | 가격: ₩ 10,600
[10/44] ✅ appid: 2161050 | 개발사: Purple Ray Studio | 배급사: Untold Tales, CouchPlay Interactive | 가격: ₩ 8,600
[11/44] ✅ appid: 1794490 | 개발사: nekogameteacher | 배급사: nekogameteacher | 가격: 무료 또는 정보 없음
[12/44] ✅ appid: 

In [73]:
steam_eng = pd.read_csv("03-3. steam_eng_updated.csv", encoding='utf-8-sig')

In [74]:
steam_eng

,appid,게임명,출시일,장르,OS,DLC여부,평가점수,사용자 평가 수,개발사,배급사,가격
0,1384160,guilty gear -strive-,2021년 06월 11일,격투 | 2D 격투 | 웅장한 사운드트랙 | 애니메이션 | PvP,Windows,0,매우 긍정적,43066,Arc System Works,Arc System Works,"₩ 19,950"
1,2141910,magic: the gathering arena,2023년 05월 24일,전략 | 카드 게임 | 트레이딩 카드 게임 | 무료 플레이 | 덱빌딩,Windows,0,복합적,21004,Wizards of the Coast LLC,Wizards of the Coast LLC,무료
2,1477590,ez2on reboot : r,2022년 04월 14일,리듬 | 액션 | 음악 | 캐주얼 | 스포츠,Windows,0,매우 긍정적,5612,"Neonovice Co., Ltd., SQUARE PIXELS","Neonovice Co., Ltd.","₩ 44,800"
3,1326470,sons of the forest,2024년 02월 23일,생존 | 오픈 월드 | 멀티플레이어 | 협동 | 생존 공포,Windows,0,매우 긍정적,234109,Endnight Games Ltd,Newnight,"₩ 32,000"
4,550,left 4 dead 2,2009년 11월 17일,좀비 | 협동 | 1인칭 슈팅 | 멀티플레이어 | 슈팅,Windows,0,압도적으로 긍정적,747304,Valve,Valve,"₩ 11,000"
...,...,...,...,...,...,...,...,...,...,...,...
18823,1723880,crabs and rats,출시 예정,액션 | 캐주얼 | 아케이드 | 플랫폼 | 2D 플랫폼,Windows,0,없음,없음,Games For Pleasure,Games For Pleasure,"₩ 10,000"
18824,1782170,adventure jam,출시 예정,어드벤처 | 액션 | RPG | 2D 격투 | 액션 어드벤처,Windows,0,없음,없음,NaN,NaN,무료
18825,346110,the train arrived!,발표 예정,어드벤처 | RPG | 액션 | 기차 | 퍼즐,Windows,0,없음,없음,"Studio Wildcard, Instinct Games, Efecto Studio...","Studio Wildcard, Snail Games USA","R$ 46,99"
18826,3010460,dispatch control,발표 예정,시뮬레이션 | 전략 | 캐주얼 | 실시간 전략 | 대전략,Windows,0,없음,없음,Hypercent Inc.,Hypercent Inc.,무료


In [76]:
steam_eng = steam_eng.drop_duplicates(subset=['게임명'], keep='first')

In [79]:
steam_eng = steam_eng[~steam_eng['출시일'].isna()]

In [83]:
steam_eng[steam_eng['개발사'].isna()]

,appid,게임명,출시일,장르,OS,DLC여부,평가점수,사용자 평가 수,개발사,배급사,가격
253,3300460,company of heroes 3: stealth & stronghold,2025년 03월 27일,전략 | 액션 | 고어 | 폭력,Windows,0,없음,없음,NaN,NaN,무료
261,3787460,warframe: valkyr heirloom collection,2025년 07월 21일,RPG | 액션 | 무료 플레이 | 고어 | 폭력,Windows,0,긍정적,41,NaN,NaN,무료
346,1194630,the dark pictures anthology: little hope,2020년 10월 30일,공포 | 복수 결말 | 풍부한 스토리 | 심리적 공포 | 협동,Windows,0,대체로 긍정적,5991,NaN,NaN,무료
354,680420,outriders,"1 Apr, 2021",약탈 슈팅 | RPG | 협동 | 3인칭 슈팅 | 액션,Windows,0,복합적,45575,NaN,NaN,무료
357,204100,max payne 3,2012년 06월 01일,액션 | 3인칭 슈팅 | 블렛타임 | 풍부한 스토리 | 멀티플레이어,Windows,0,매우 긍정적,49044,NaN,NaN,무료
...,...,...,...,...,...,...,...,...,...,...,...
18673,208803,tomb raider: headshot reticule,2013년 03월 26일,액션 | 어드벤처,Windows,0,복합적,47,NaN,NaN,무료
18674,232530,biohazard 6: onslaught mode,2013년 03월 22일,액션 | 어드벤처,Windows,0,대체로 긍정적,68,NaN,NaN,무료
18683,3203390,call of duty?: black ops ii season pass,2013년 03월 01일,액션 | 1인칭 슈팅 | 멀티플레이어 | 좀비,Windows,0,없음,없음,NaN,NaN,무료
18769,2369456,far cry? 2,2008년 10월 23일,오픈 월드 | 1인칭 슈팅 | 액션 | 슈팅 | 싱글 플레이어,Windows,0,대체로 긍정적,15786,NaN,NaN,무료


In [84]:
steam_eng.to_csv('최종_스팀게임정보.csv', encoding='utf-8-sig')

In [ ]:
missing_appids = steam_eng[steam_eng['개발사'].isna() | steam_eng['배급사'].isna()]['appid'].dropna().astype(int).tolist()

In [ ]:
import requests
import pandas as pd
import numpy as np
import time

developers = []
publishers = []
collected_ids = []

total = len(missing_appids)
print(f"🚀 {total}개 appid 개발사/배급사 수집 시작")

for idx, appid in enumerate(missing_appids, 1):
    try:
        url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
        response = requests.get(url, timeout=5)
        data = response.json()
        app_data = data.get(str(appid))

        if app_data and app_data.get('success') and app_data.get('data'):
            info = app_data['data']
            dev = ", ".join(info.get('developers', [])) or np.nan
            pub = ", ".join(info.get('publishers', [])) or np.nan
        else:
            dev, pub = np.nan, np.nan

        developers.append(dev)
        publishers.append(pub)
        collected_ids.append(appid)
        print(f"[{idx}/{total}] ✅ appid: {appid} | 개발사: {dev} | 배급사: {pub}")

    except Exception as e:
        print(f"[{idx}/{total}] ⚠️ appid: {appid} 실패 - {e}")
        developers.append(np.nan)
        publishers.append(np.nan)
        collected_ids.append(appid)

    time.sleep(0.2)

🚀 628개 appid 개발사/배급사 수집 시작
[1/628] ✅ appid: 401920 | 개발사: Nicalis, Inc., Edmund McMillen | 배급사: nan
[2/628] ✅ appid: 3796620 | 개발사: Behaviour Interactive Inc. | 배급사: nan
[3/628] ✅ appid: 3300460 | 개발사: Relic Entertainment | 배급사: Relic Entertainment
[4/628] ✅ appid: 3787460 | 개발사: Digital Extremes | 배급사: Digital Extremes
[5/628] ✅ appid: 1194630 | 개발사: Supermassive Games | 배급사: BANDAI NAMCO Entertainment Europe
[6/628] ✅ appid: 2439690 | 개발사: 10tons Ltd | 배급사: nan
[7/628] ✅ appid: 680420 | 개발사: People Can Fly | 배급사: Square Enix
[8/628] ✅ appid: 204100 | 개발사: Rockstar Studios | 배급사: Rockstar Games
[9/628] ✅ appid: 667530 | 개발사: Oleg Skutte | 배급사: nan
[10/628] ✅ appid: 3042220 | 개발사: System Era Softworks | 배급사: nan
[11/628] ✅ appid: 261640 | 개발사: 2K Australia, Gearbox Software, Aspyr (Linux) | 배급사: 2K, Aspyr (Linux)
[12/628] ✅ appid: 2317930 | 개발사: Marco Amadei | 배급사: nan
[13/628] ✅ appid: 35130 | 개발사: Crystal Dynamics | 배급사: Crystal Dynamics
[14/628] ✅ appid: 625500 | 개발사: Redemption Roa

In [ ]:
result_df = pd.DataFrame({
    'appid': collected_ids,
    '개발사_수정': developers,
    '배급사_수정': publishers
})

# appid 기준 병합
steam_eng = steam_eng.merge(result_df, on='appid', how='left')

# 기존이 NaN인 항목만 업데이트
steam_eng['개발사'] = steam_eng['개발사'].fillna(steam_eng['개발사_수정'])
steam_eng['배급사'] = steam_eng['배급사'].fillna(steam_eng['배급사_수정'])

# 임시 열 제거
steam_eng.drop(columns=['개발사_수정', '배급사_수정'], inplace=True)

steam_eng


,appid,게임명,출시일,장르,OS,DLC여부,평가점수,사용자 평가 수,개발사,배급사,가격
0,1384160,guilty gear -strive-,2021년 06월 11일,격투 | 2D 격투 | 웅장한 사운드트랙 | 애니메이션 | PvP,Windows,0,매우 긍정적,43066,Arc System Works,Arc System Works,"₩ 19,950"
1,2141910,magic: the gathering arena,2023년 05월 24일,전략 | 카드 게임 | 트레이딩 카드 게임 | 무료 플레이 | 덱빌딩,Windows,0,복합적,21004,Wizards of the Coast LLC,Wizards of the Coast LLC,무료
2,1477590,ez2on reboot : r,2022년 04월 14일,리듬 | 액션 | 음악 | 캐주얼 | 스포츠,Windows,0,매우 긍정적,5612,"Neonovice Co., Ltd., SQUARE PIXELS","Neonovice Co., Ltd.","₩ 44,800"
3,1326470,sons of the forest,2024년 02월 23일,생존 | 오픈 월드 | 멀티플레이어 | 협동 | 생존 공포,Windows,0,매우 긍정적,234109,Endnight Games Ltd,Newnight,"₩ 32,000"
4,550,left 4 dead 2,2009년 11월 17일,좀비 | 협동 | 1인칭 슈팅 | 멀티플레이어 | 슈팅,Windows,0,압도적으로 긍정적,747304,Valve,Valve,"₩ 11,000"
...,...,...,...,...,...,...,...,...,...,...,...
18837,1723880,crabs and rats,출시 예정,액션 | 캐주얼 | 아케이드 | 플랫폼 | 2D 플랫폼,Windows,0,없음,없음,Games For Pleasure,Games For Pleasure,"₩ 10,000"
18838,1782170,adventure jam,출시 예정,어드벤처 | 액션 | RPG | 2D 격투 | 액션 어드벤처,Windows,0,없음,없음,NEOWIZ,NEOWIZ,무료
18839,346110,the train arrived!,발표 예정,어드벤처 | RPG | 액션 | 기차 | 퍼즐,Windows,0,없음,없음,"Studio Wildcard, Instinct Games, Efecto Studio...","Studio Wildcard, Snail Games USA","R$ 46,99"
18840,3010460,dispatch control,발표 예정,시뮬레이션 | 전략 | 캐주얼 | 실시간 전략 | 대전략,Windows,0,없음,없음,Hypercent Inc.,Hypercent Inc.,무료


In [89]:
steam_eng = steam_eng[~steam_eng['개발사'].isna()]

In [108]:
steam_eng.to_csv("03-3. steam_eng_updated.csv", index=False, encoding='utf-8-sig')
steam_eng.to_csv("최종_스팀게임정보.csv", index=False, encoding='utf-8-sig')